Create a simple agent with a simple graph

In [0]:
%pip install -U -qqq mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents uv
dbutils.library.restartPython()

### Import Libraries

In [0]:
from databricks_langchain import ChatDatabricks

from pprint import pprint
from pydantic import BaseModel

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode, tools_condition

from IPython.display import Image, display
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END

### Define Model

In [0]:
# define chat model with an llm
chat_model = ChatDatabricks(endpoint="databricks-claude-3-7-sonnet")

### Create Tools

In [0]:
# define tools
def multiply(a: int, b: int) -> int:
    "This is a function to multiply two values"
    return a * b

def add(a: int, b: int) -> int:
    "This is a function to sum two values"
    return a + b

def divide(a: int, b: int) -> float:
    "This is a function to divide two values"
    return a / b

tools = [multiply, add, divide]

llm_with_tools = chat_model.bind_tools(tools)

In [0]:
system_message = SystemMessage(content = "You are a helpful assistant that responds on Azure Cloud engineering and Databricks engineering question and perform arithmetic on a set of inputs.")

### Define Graph

In [0]:
#Node
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([system_message] +  state["messages"])]}

#Build Graph
builder = StateGraph(MessagesState)

#define nodes to do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

#define edges: determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant", 
    #If the latest message (result) from assistant is a tool then it routes to the tool, if not it routes to END
    tools_condition
)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

display(Image(react_graph.get_graph().draw_mermaid_png()))

### Call Agent

In [0]:
from langchain_core.messages import HumanMessage
messages = [
            HumanMessage(content="Multiply 3 and 5, to the output add 10 and the outbut divide by 5. Then explain in simple terms what is Databricks Adaptive Query Execution.")
            ]
            
messages = react_graph.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()